In [1]:
import pandas as pd
import numpy as np

In [5]:
#this is the original dataset
df = pd.read_csv(r"C:\Users\nalab\Desktop\Lectures\STAT362/kfr-incidents.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28006 entries, 0 to 28005
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Incident #              28006 non-null  object 
 1   Alarm Time              28006 non-null  object 
 2   Call Type               27969 non-null  object 
 3   Incident Type           27969 non-null  object 
 4   Response Time (Median)  27603 non-null  float64
 5   Incident Duration       28006 non-null  float64
 6   Stations                27439 non-null  object 
 7   Units                   28003 non-null  object 
 8   Unit Count              28006 non-null  int64  
 9   GeoJSON                 27352 non-null  object 
 10  geo_point_2d            27352 non-null  object 
dtypes: float64(2), int64(1), object(8)
memory usage: 2.4+ MB


In [13]:
df.head()

,Incident #,Alarm Time,Call Type,Incident Type,Response Time (Median),Incident Duration,Stations,Units,Unit Count,GeoJSON,geo_point_2d
0,16-0100010,2016-01-01T00:00:00-05:00,Medical,"Vital signs absent, DOA",3.48,9.03,PALACE RD,P261,1,"{""coordinates"": [-76.52414817403951, 44.248794...","44.24879457292734,-76.52414817403951"
1,16-0100012,2016-01-01T00:00:00-05:00,Fire,Fire,5.95,19.10,RAILWAY ST,P251,1,"{""coordinates"": [-76.48215423062096, 44.270774...","44.27077492953324,-76.48215423062096"
2,16-0100013,2016-01-01T00:00:00-05:00,Fire,Pot on Stove (no fire),2.37,18.87,"BROCK ST,PALACE RD","L241,P241,P261",3,"{""coordinates"": [-76.49005223242744, 44.232263...","44.232263038895105,-76.49005223242744"
3,16-0100031,2016-01-03T00:00:00-05:00,False Fire Call,Human - Perceived Emergency,3.50,12.67,"BROCK ST,PALACE RD,RAILWAY ST","C241,L241,P241,P251,P261",5,"{""coordinates"": [-76.4909987107299, 44.2375423...","44.23754236048484,-76.4909987107299"
4,16-0100036,2016-01-03T00:00:00-05:00,False Fire Call,"Human - Malicious intent, prank",3.82,12.13,"BROCK ST,RAILWAY ST,WOODBINE RD","L241,P241,P251,P271",4,"{""coordinates"": [-76.49709302509763, 44.240269...","44.240269547664106,-76.49709302509763"


In [8]:
#my first task is calculate the distance from the station to the incident
#however, as you can see, sometimes more than one station was called to the incident.
#So I will need to calcaulte the distance for each station that was called to the incident for each incident
df.head()
df2 = (df.assign(Stations=df.Stations.str.split(','))
    .explode('Stations'))

In [14]:
stations = df2["Stations"].unique().tolist()
del stations[-2]
stations

['PALACE RD',
 'RAILWAY ST',
 'BROCK ST',
 'WOODBINE RD',
 'DAYS RD',
 'GORE RD',
 'UNITY RD',
 'BREWERS MILLS RD',
 'JOYCEVILLE RD',
 '38 HY',
 "O'CONNOR DR"]

In [15]:
#These are the coordinates for the fire stations mentioned above.
coords = []
coords.append([44.232516043029975, -76.5109469106995]) 
coords.append([44.24840106149649, -76.49360694105341]) 
coords.append([44.23208442829623, -76.48937740612082])
coords.append([44.25868441024724, -76.58473481221753])
coords.append([44.21983094913333, -76.56735302098983])
coords.append([44.25938097934096, -76.45590489769965])
coords.append([44.320427544582614, -76.50095095803893])
coords.append([44.411746552256936, -76.28037648126949])
coords.append([44.30315195737124, -76.3279602283708])
coords.append([44.49093752591384, -76.6877344282853])
coords.append([44.25998977937879, -76.5654337971918])
len(coords)

11

In [18]:
station_coords = pd.DataFrame(
    {'Stations': stations,
     'station_coords': coords})

In [20]:
df2 = df2.merge(station_coords, on='Stations')
df2 = df2.dropna()
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44377 entries, 0 to 46019
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Incident #              44377 non-null  object 
 1   Alarm Time              44377 non-null  object 
 2   Call Type               44377 non-null  object 
 3   Incident Type           44377 non-null  object 
 4   Response Time (Median)  44377 non-null  float64
 5   Incident Duration       44377 non-null  float64
 6   Stations                44377 non-null  object 
 7   Units                   44377 non-null  object 
 8   Unit Count              44377 non-null  int64  
 9   GeoJSON                 44377 non-null  object 
 10  geo_point_2d            44377 non-null  object 
 11  station_coords          44377 non-null  object 
dtypes: float64(2), int64(1), object(9)
memory usage: 4.4+ MB


In [25]:
split = df2['geo_point_2d'].str.split(',', expand =True)
split.columns = ['lat', 'lon']
split = split.astype(float)

df2 = pd.concat([df2,split], axis=1)
df2.head()

,Incident #,Alarm Time,Call Type,Incident Type,Response Time (Median),Incident Duration,Stations,Units,Unit Count,GeoJSON,geo_point_2d,station_coords,lat,lon
0,16-0100010,2016-01-01T00:00:00-05:00,Medical,"Vital signs absent, DOA",3.48,9.03,PALACE RD,P261,1,"{""coordinates"": [-76.52414817403951, 44.248794...","44.24879457292734,-76.52414817403951","[44.232516043029975, -76.5109469106995]",44.248795,-76.524148
1,16-0100013,2016-01-01T00:00:00-05:00,Fire,Pot on Stove (no fire),2.37,18.87,PALACE RD,"L241,P241,P261",3,"{""coordinates"": [-76.49005223242744, 44.232263...","44.232263038895105,-76.49005223242744","[44.232516043029975, -76.5109469106995]",44.232263,-76.490052
2,16-0100031,2016-01-03T00:00:00-05:00,False Fire Call,Human - Perceived Emergency,3.50,12.67,PALACE RD,"C241,L241,P241,P251,P261",5,"{""coordinates"": [-76.4909987107299, 44.2375423...","44.23754236048484,-76.4909987107299","[44.232516043029975, -76.5109469106995]",44.237542,-76.490999
3,16-0100041,2016-01-03T00:00:00-05:00,False Fire Call,Alarm System Equipment - Malfunction,4.72,16.03,PALACE RD,"L241,P241,P261",3,"{""coordinates"": [-76.51224553280572, 44.250304...","44.25030436167093,-76.51224553280572","[44.232516043029975, -76.5109469106995]",44.250304,-76.512246
5,16-0100060,2016-01-04T00:00:00-05:00,Fire,"Overheat (no fire, e.g. engines, mechanical de...",3.03,16.62,PALACE RD,"P251,P261",2,"{""coordinates"": [-76.481897924849, 44.23513009...","44.235130093607815,-76.481897924849","[44.232516043029975, -76.5109469106995]",44.235130,-76.481898


In [32]:
from geopy import distance
def f(x):    
    return distance.distance((x[11][0],x[11][1]),(x[12],x[13])).km 

distance = df2.apply(f, axis=1)

In [35]:
df2["distance"] = distance

In [42]:
#here I have the distance from each station to each incident. I got the minimum distance,
#which is essentially the distance of the closest station to the incident
#I also calculated the average distance of the stations that were called in to the accident
#I also created a new column called latitude and longitude, which are the coordinates of the incident
grouped = df2.groupby(['Incident #'], as_index=False).agg({'distance': 'min','lat':'first','lon':'first'})
grouped["distance_average"] = df2.groupby(['Incident #'], as_index=False)['distance'].mean()['distance']
grouped.head(10)

,Incident #,distance,lat,lon,distance_average
0,16-0100001,0.428257,44.244573,-76.492985,0.428257
1,16-0100002,0.253464,44.233512,-76.491852,0.253464
2,16-0100003,0.253464,44.233512,-76.491852,0.253464
3,16-0100004,1.525518,44.223026,-76.524744,1.525518
4,16-0100006,4.795184,44.296938,-76.426320,9.980482
5,16-0100010,2.093770,44.248795,-76.524148,2.093770
6,16-0100011,2.581805,44.271074,-76.486536,3.460053
7,16-0100012,2.649007,44.270775,-76.482154,2.649007
8,16-0100013,0.057451,44.232263,-76.490052,0.863507
9,16-0100014,1.616241,44.260123,-76.604875,1.616241


In [48]:
#here I created one hot encoding for the fire stations
df3 = pd.crosstab(df2['Incident #'],df2['Stations']).rename_axis(None,axis=1).add_prefix('station_')
#df3 = df.merge(grouped, on='Incident #')
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26389 entries, 16-0100001 to 22-0400255
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   station_38 HY             26389 non-null  int64
 1   station_BREWERS MILLS RD  26389 non-null  int64
 2   station_BROCK ST          26389 non-null  int64
 3   station_DAYS RD           26389 non-null  int64
 4   station_GORE RD           26389 non-null  int64
 5   station_JOYCEVILLE RD     26389 non-null  int64
 6   station_O'CONNOR DR       26389 non-null  int64
 7   station_PALACE RD         26389 non-null  int64
 8   station_RAILWAY ST        26389 non-null  int64
 9   station_UNITY RD          26389 non-null  int64
 10  station_WOODBINE RD       26389 non-null  int64
dtypes: int64(11)
memory usage: 2.4+ MB


In [49]:
df3.head()

,station_38 HY,station_BREWERS MILLS RD,station_BROCK ST,station_DAYS RD,station_GORE RD,station_JOYCEVILLE RD,station_O'CONNOR DR,station_PALACE RD,station_RAILWAY ST,station_UNITY RD,station_WOODBINE RD
Incident #,,,,,,,,,,,
16-0100001,0,0,0,0,0,0,0,0,1,0,0
16-0100002,0,0,1,0,0,0,0,0,0,0,0
16-0100003,0,0,1,0,0,0,0,0,0,0,0
16-0100004,0,0,0,0,0,0,0,1,0,0,0
16-0100006,0,1,0,0,1,1,0,0,0,0,0


In [82]:
df4 = df.merge(grouped, on='Incident #')
df4 = df4.merge(df3, on='Incident #')
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26391 entries, 0 to 26390
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Incident #                26391 non-null  object 
 1   Alarm Time                26391 non-null  object 
 2   Call Type                 26391 non-null  object 
 3   Incident Type             26391 non-null  object 
 4   Response Time (Median)    26391 non-null  float64
 5   Incident Duration         26391 non-null  float64
 6   Stations                  26391 non-null  object 
 7   Units                     26391 non-null  object 
 8   Unit Count                26391 non-null  int64  
 9   GeoJSON                   26389 non-null  object 
 10  geo_point_2d              26389 non-null  object 
 11  distance                  26391 non-null  float64
 12  lat                       26391 non-null  float64
 13  lon                       26391 non-null  float64
 14  distan

In [85]:
from datetime import datetime
#here I created a new column called day of the year.
#This is basically taking the day of the year instead of the date. So Febraury 02 turns into 33.
#This is better because we cannot use date for regression.

df4["Alarm Time"] = df4["Alarm Time"].astype(str)
df4["Alarm Time"] = df4["Alarm Time"].str[:10]
df4["Alarm Time"] = pd.to_datetime(df4["Alarm Time"], format='%Y-%m-%d')
df4['day_of_year'] = df4['Alarm Time'].dt.dayofyear

In [86]:
df4.info(40)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26391 entries, 0 to 26390
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Incident #                26391 non-null  object        
 1   Alarm Time                26391 non-null  datetime64[ns]
 2   Call Type                 26391 non-null  object        
 3   Incident Type             26391 non-null  object        
 4   Response Time (Median)    26391 non-null  float64       
 5   Incident Duration         26391 non-null  float64       
 6   Stations                  26391 non-null  object        
 7   Units                     26391 non-null  object        
 8   Unit Count                26391 non-null  int64         
 9   GeoJSON                   26389 non-null  object        
 10  geo_point_2d              26389 non-null  object        
 11  distance                  26391 non-null  float64       
 12  lat               

In [87]:
#Here I did one hot encoding for call type.
#I also dropped the following columns:
#'Call Type', 'Incident Type','Alarm Time','Stations','Units','GeoJSON','geo_point_2d','AlarmDate', 'Incident #'
one_hot = pd.get_dummies(df4['Call Type'])
df5 = df4.drop(['Call Type', 'Incident Type','Alarm Time','Stations','Units','GeoJSON','geo_point_2d',
               'AlarmDate', 'Incident #'],axis = 1)
df5 = df5.join(one_hot)
df5.head()

,Response Time (Median),Incident Duration,Unit Count,distance,lat,lon,distance_average,station_38 HY,station_BREWERS MILLS RD,station_BROCK ST,...,day_of_year,CO Alarm,False Fire Call,Fire,Medical,Motor Vehicle Accident,Other,Other Rescue,Structure Rescue,Water Rescue
0,3.48,9.03,1,2.093770,44.248795,-76.524148,2.093770,0,0,0,...,1,0,0,0,1,0,0,0,0,0
1,5.95,19.10,1,2.649007,44.270775,-76.482154,2.649007,0,0,0,...,1,0,0,1,0,0,0,0,0,0
2,2.37,18.87,3,0.057451,44.232263,-76.490052,0.863507,0,0,1,...,1,0,0,1,0,0,0,0,0,0
3,3.50,12.67,5,0.620145,44.237542,-76.490999,1.177753,0,0,1,...,3,0,1,0,0,0,0,0,0,0
4,3.82,12.13,4,0.945484,44.240270,-76.497093,3.112337,0,0,1,...,3,0,1,0,0,0,0,0,0,0


In [89]:
df5.to_csv(r"C:\Users\nalab\Desktop\Lectures\STAT362/regression_ready.csv")